In [1]:
import pandas as pd
from stage import fullPrint, getStage
from datetime import datetime, timedelta
import yahoo_fin.stock_info as yf
import numpy as np
import pickle
import swifter
import os

PEAK = 13
allList = []
nasdaqList = pd.read_pickle("stockData/tickerList.pkl")
allList = (nasdaqList)

SectorDict = {
    "Energy":"XLE",
    "Technology":"XLK",
    "Communication Services":"XLC",
    "Consumer Defensive":"XLP",
    "Healthcare":"XLV",
    "Consumer Cyclical":"XLY",
    "Industrials":"XLI",
    "Utilities":"XLU",
    "Basic Materials":"XLB",
    "Financial Services":"XLF",
    "Real Estate":"XLRE"}
spList = yf.tickers_sp500()
spList = set(spList)
dictionary = {}
i = 0
allLen = str(len(nasdaqList))
stockDataTickerList = []
now = datetime.now()

In [2]:
for ticker in nasdaqList:
    try:
        df = yf.get_data(ticker, start_date="1995-01-06",end_date = now, index_as_date = True)
        df = df[~df.index.duplicated()]
        dfBuy = df
        df['Dates'] = pd.to_datetime(df.index)
        df = df[df['Dates'].dt.weekday == 4]
        df = df.drop('Dates', axis = 1)
        df.to_pickle('stockData/nyseNasdaq/'+ticker+'.pkl')
        industry = yf.get_company_info(ticker).at['sector','Value']
        dictionary[ticker] = SectorDict[industry]
        dfBuy['Dates'] = pd.to_datetime(dfBuy.index)
        dfBuy = dfBuy[dfBuy['Dates'].dt.weekday == 0]
        dfBuy = dfBuy.drop('Dates', axis = 1)
        dfBuy.to_pickle('stockData/nyseNasdaq/'+ticker+'Buy.pkl')
        stockDataTickerList.append(ticker)
    except:
        print(ticker+" not found")
    i = i+1
    print(str(i)+'/'+allLen)
try:
    sectorDict = open('stockData/nasdaq.pkl', 'wb')
    pickle.dump(dictionary, sectorDict)
    sectorDict.close()
except:
    print("Something went wrong")

1/4040
2/4040
3/4040
4/4040
5/4040
6/4040
7/4040
8/4040
9/4040
10/4040
11/4040
12/4040
13/4040
14/4040
15/4040
16/4040
17/4040
18/4040
19/4040
20/4040
21/4040
22/4040
23/4040
24/4040
25/4040
26/4040
27/4040
28/4040
29/4040
30/4040
31/4040
32/4040
33/4040
34/4040
35/4040
36/4040
37/4040
38/4040
39/4040
40/4040
41/4040
42/4040
43/4040
44/4040
45/4040
46/4040
47/4040
48/4040
49/4040
50/4040
51/4040
52/4040
53/4040
54/4040
55/4040
56/4040
57/4040
58/4040
59/4040
60/4040
61/4040
62/4040
63/4040
64/4040
65/4040
66/4040
67/4040
68/4040
69/4040
70/4040
71/4040
72/4040
73/4040
74/4040
75/4040
76/4040
77/4040
78/4040
79/4040
80/4040
81/4040
82/4040
83/4040
84/4040
85/4040
86/4040
87/4040
88/4040
89/4040
90/4040
91/4040
92/4040
93/4040
94/4040
95/4040
96/4040
97/4040
98/4040
99/4040
100/4040
101/4040
102/4040
103/4040
104/4040
105/4040
106/4040
107/4040
108/4040
109/4040
110/4040
111/4040
112/4040
113/4040
114/4040
115/4040
116/4040
117/4040
118/4040
119/4040
120/4040
121/4040
122/4040
123/4040
1

In [3]:
weights = np.arange(1,31)
sumWeights = np.sum(weights)
spList = yf.tickers_sp500()
def calculateSlopeOfTicker(df,index):
    if index < 31:
        return 0
    else:
        return df.iat[index,4]/df.iat[index-1,4]

def calculateSpyDfSlopeOfTicker(df,index):
    if index < 31:
        return 0
    else:
        return df.iat[index,4]/df.iat[index-1,8]

def product(df,index):
    if index < 30:
        return 0
    else:
        return df.iat[index,0]/df.iat[index-30,0]
def calculatePeak(df,index, diff, col):
    if index == 0:
        return df.iat[index,0]
    if index < diff:
        return max(df.iat[index-1,col],df.iat[index,0])


In [4]:
spDf = yf.get_data('^GSPC', start_date="1995-01-06",end_date = now, index_as_date = True)
spDf = spDf[~spDf.index.duplicated()]
spDf['close'] = spDf['close'].rolling(window=5).apply(lambda x: np.average(x))
spDf['Dates'] = pd.to_datetime(spDf.index)
spDf = spDf[spDf['Dates'].dt.weekday == 4]
spDf = spDf.drop('Dates', axis = 1)
spDf['percent'] = spDf.swifter.apply(lambda x: product(spDf,spDf.index.get_loc(x.name)),axis=1)
spDf['WMA30'] = spDf['close'].rolling(window=30).apply(lambda x: np.sum(weights*x)/sumWeights)
spDf['WMA30Slope'] = spDf.swifter.apply(lambda x: calculateSpyDfSlopeOfTicker(spDf,spDf.index.get_loc(x.name)),axis=1)
spDf.to_pickle("stockData/Spy.pkl")

Pandas Apply: 100%|██████████| 1377/1377 [00:00<00:00, 24449.08it/s]


In [5]:
for ticker in stockDataTickerList:
    try:
        stockDf = pd.read_pickle('stockData/nyseNasdaq/'+ticker+'.pkl')
        buyDf = pd.read_pickle('stockData/nyseNasdaq/'+ticker+'Buy.pkl')
        if stockDf['close'].isnull().values.any() or buyDf['open'].isnull().values.any():
            os.remove('stockData/nyseNasdaq/'+ticker+'.pkl')
            os.remove('stockData/nyseNasdaq/'+ticker+'Buy.pkl')
        stockDf['fiveYearHigh'] = stockDf['close'].rolling(window=260).max().shift(1).fillna(0)  
        stockDf['WMA30'] = stockDf['close'].rolling(window=30).apply(lambda x: np.sum(weights*x)/sumWeights)
        stockDf['WMA30Slope'] = stockDf.swifter.apply(lambda x: calculateSlopeOfTicker(stockDf,stockDf.index.get_loc(x.name)),axis=1)
        stockDf['volumePerc'] = stockDf['volume'].pct_change()
        stockDf['percent'] = stockDf.swifter.apply(lambda x: product(stockDf,stockDf.index.get_loc(x.name)),axis=1)
        stockDf['RS'] = stockDf['percent'] - spDf['percent']
        #stockDf = stockDf.dropna()
        stockDf['peak'] = 0
        stockDf['trough'] = 0
        stockDf['support'] = 0
        stockDf['initialSupport'] = 0
        for index, element in stockDf.iterrows():
            numIndex =stockDf.index.get_loc(index)
            if numIndex < 260:
                stockDf.iat[numIndex, PEAK] = calculatePeak(stockDf, numIndex, 260, 3)
        stockDf['Stage'] = ""
        stockDf['secondBuy'] = False
        stockDf['resistance'] = 0
        stockDf['shortPeak'] = 0
        stockDf['shortTrough'] = 0
        print(ticker)
        stockDf.to_pickle('stockData/nyseNasdaq/'+ticker+'.pkl')
    except:
        pass


Pandas Apply: 100%|██████████| 717/717 [00:00<00:00, 23795.07it/s]


AACG


Pandas Apply: 100%|██████████| 24/24 [00:00<00:00, 19718.57it/s]


AACI


Pandas Apply: 100%|██████████| 37/37 [00:00<00:00, 8178.62it/s]


...

SCHN


Pandas Apply: 100%|██████████| 62/62 [00:00<00:00, 16003.87it/s]


SCOA


Pandas Apply: 100%|██████████| 403/403 [00:00<00:00, 9820.90it/s]

SCYX


In [6]:
transactionTemplate = yf.get_data('AAPL', start_date="1995-01-02",end_date= now, index_as_date = True, interval="1wk").drop(['open','high','low','close','adjclose','volume','ticker'],axis=1)
transactionTemplate = transactionTemplate[~transactionTemplate.index.duplicated()]
transactionTemplate.to_pickle("transactionTemplate.pkl")

In [7]:
for ticker in nasdaqList:
    try:
        stockDf = pd.read_pickle('stockData/nyseNasdaq/'+ticker+'.pkl')
        stockDf = stockDf.drop(['open','high','low','adjclose'],axis=1)
        stockDf.to_pickle('stockData/nyseNasdaq/'+ticker+'.pkl')
    except:
        pass

In [8]:
now = datetime.now()
monday = now - timedelta(days = now.weekday())
now = now.strftime('%Y-%m-%d')
df = yf.get_data(ticker, start_date="1995-01-06",end_date = now, index_as_date = True)
df['Dates'] = pd.to_datetime(df.index)
df = df[df['Dates'].dt.weekday == 4]
df = df.drop('Dates', axis = 1)
df

,open,high,low,close,adjclose,volume,ticker
2014-05-02,97.500000,98.400002,86.300003,90.000000,90.000000,85640,SCYX
2014-05-09,89.199997,93.000000,87.000000,93.000000,93.000000,6030,SCYX
2014-05-16,90.300003,90.300003,89.699997,90.000000,90.000000,1190,SCYX
2014-05-23,89.500000,94.599998,89.500000,90.099998,90.099998,1120,SCYX
2014-05-30,84.000000,85.000000,82.500000,84.000000,84.000000,7000,SCYX
...,...,...,...,...,...,...,...
2022-04-01,3.960000,3.960000,3.660000,3.870000,3.870000,389900,SCYX
2022-04-08,3.660000,3.660000,3.490000,3.590000,3.590000,109000,SCYX
2022-04-22,2.560000,2.630000,2.300000,2.330000,2.330000,3133400,SCYX
2022-04-29,2.550000,2.720000,2.510000,2.670000,2.670000,473100,SCYX


In [9]:
transactionTemplate = yf.get_data('AAPL', start_date="1995-01-06",end_date= now, index_as_date = True).drop(['open','high','low','close','adjclose','volume','ticker'],axis=1)
transactionTemplate['Dates'] = pd.to_datetime(transactionTemplate.index)
transactionTemplate = transactionTemplate[transactionTemplate['Dates'].dt.weekday == 4]
transactionTemplate = transactionTemplate.drop('Dates', axis = 1)
transactionTemplate.to_pickle("transactionTemplate.pkl")

In [2]:
SectorDict = {
    "Energy":"XLE","Technology":"XLK",
    "Communication Services":"XLC",
    "Consumer Staples":"XLP",
    "Health Care":"XLV",
    "Consumer Cyclical":"XLY",
    "Industrial":"XLI",
    "Utilities":"XLU",
    "Materials":"XLB",
    "Financial":"XLF",
    "Real Estate":"XLRE"}
sectorDfList = []
for sector in SectorDict.values():
    sectorDfList.append(pd.read_pickle("stockData/sectorCharts/"+sector+".pkl"))
goodSectorDf = sectorDfList[0].drop(['open','high','low','adjclose','close','ticker','volume','percent','MA30','MA30Slope','RS'],axis=1)
goodSectorDf['Sectors'] = [[] for _ in range(len(goodSectorDf))]
goodSectorDf.index = sectorDfList[0].index
index = 0
for date in sectorDfList[0].index:
    listOfSector = []
    for df in sectorDfList:
        try:
            if  df.at[date,'MA30Slope'] > 0.992549883242513 and df.loc[date,'RS'] > -0.3044455098712273:
                if df.at[date,'close'] > df.at[date,'MA30']:
                    #print(df.at[date,'ticker'])
                    listOfSector.append(df.at[date,'ticker'])
        except: 
            pass
        # print(listOfSector)
    goodSectorDf.iat[index,0] = set(listOfSector)
    index = index + 1


In [3]:
removeList = []
CLOSE = 0
TICKER = 2
for ticker in nasdaqList:
    try:
        df = pd.read_pickle("stockData/nyseNasdaq/"+ticker+".pkl")
        lastClose = df.iat[0,CLOSE]
        isFirst = True
        index = 0
        for date in df.Stage:
            if isFirst == True:
                isFirst = False
                continue
            if (df.iat[index,CLOSE] >= lastClose * 5):
                removeList.append(ticker)
                break
            lastClose = df.iat[index,CLOSE]
            index += 1
        stockDf = pd.read_pickle('stockData/nyseNasdaq/'+ticker+'.pkl')
        buyDf = pd.read_pickle('stockData/nyseNasdaq/'+ticker+'Buy.pkl')
        if stockDf['close'].isnull().values.any() or buyDf['open'].isnull().values.any():
            print(ticker)
            removeList.append(ticker)
            os.remove('stockData/nyseNasdaq/'+ticker+'.pkl')
            os.remove('stockData/nyseNasdaq/'+ticker+'Buy.pkl')
        df = getStage(ticker,[0.03361077383204303, 1.7466372328111242, -0.08498841770875612, 0.926192276854965, 1.1764676012556523, 1.1405501342698643, 0.8517616225271734, 0.8691969716399011, 1.268490629719207, 1.0013386634337298, 0.07551020458494859, 0.9489349345040117, 0.996709764664632, 0.9485725801058837, 0.9857019510456804],goodSectorDf)
        if df.empty:
            removeList.append(ticker)
    except:
        removeList.append(ticker)
print(removeList)

['BFRA', 'BWCAU', 'CHPM', 'CHPMU', 'DISCK', 'ECOL', 'FSV', 'FWRD', 'MACU', 'MBNKP', 'MBTC', 'MCAFU', 'SGMS', 'SMMT', 'STXB', 'SV', 'SVSVU', 'TUGC', 'TUGCU', 'VOR', 'VPCBU', 'VTAQU']


In [4]:
for ticker in removeList:
    nasdaqList.remove(ticker)
with open('stockData/tickerList.pkl', 'wb') as f:
    pickle.dump(nasdaqList, f)